# Introduction
This AI will create a model to determine a NSFW content, which is producted by a Stable Diffusion Model. Untill now there are only bad NSFW detectors. We will try to make one model specific, by creating safe and not safe content from the SD Model, classifying it manualy and creating this ML model to classify the content. It should be applicable to any model.

## Start the SD Server

In [ ]:
import pandas as pd
import json
import base64
import time
import requests

# helper to precassify the images
from help_nsfw_detector import predict
model = predict.load_model('./help_nsfw_detector/model/saved_model.h5')

# move files
import os
import shutil

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\rybak\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-b052b54b73bc>", line 9, in <module>
    model = predict.load_model('./help_nsfw_detector/model/saved_model.h5')
  File "C:\Users\Rybak\Desktop\react\PicGenApp\backEnd\nsfw_detector\customDetector\help_nsfw_detector\predict.py", line 55, in load_model
    model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer},compile=False)
AttributeError: module 'tensorflow' has no attribute 'keras'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\rybak\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'AttributeError' object

## Load Prompts

In [5]:
SafePosPrompt = pd.read_csv('safe pos prompts.csv', delimiter=";")
NotSafePosPrompt = pd.read_csv('not safe pos prompts.csv', delimiter=";")
NegPrompt = pd.read_csv('neg prompt.csv', delimiter=";")

#for row in range(len(SafePosPrompt)):
#    print(SafePosPrompt.at[row, 'prompt'])
len(SafePosPrompt)

73718

## Setup the Server

In [6]:
url = "http://192.168.0.109:7861"
# this will change the model on the backend server
option_payload = {
    #"sd_model_checkpoint": "AnythingV5_v32.safetensors", # switch model
    "sd_model_checkpoint": "animeModel_UpdateV10ImUsing.safetensors", # switch model
}

response = requests.post(url=f'{url}/sdapi/v1/options', json=option_payload)

print(response)

<Response [200]>


## Generate Pictures

### Helper Defs

In [7]:
def submit_post(url: str, data: dict):
    """
    Submit a POST request to the given URL with the given data.
    """
    return requests.post(url, data=json.dumps(data))
def save_encoded_image(b64_image: str, output_path: str):
    filename = time.strftime("%Y%m%d-%H%M%S") + "_picture.png"
    full_path = output_path + filename
    """
    Save the given image to the given output path.
    """
    with open(full_path, "wb") as image_file:
        image_file.write(base64.b64decode(b64_image))
    return full_path
def nsfw_detector(image_path:str):
    predict_df = predict.classify(model, [image_path])
    hentai = predict_df[image_path]["hentai"]
    porn = predict_df[image_path]["porn"]
    if porn > 0.01 or hentai > 0.01:
        return True # explicit content
    else:
        return False # not explicit content

In [ ]:
txt2img_url = 'http://192.168.0.109:7861/sdapi/v1/txt2img'
print(str(len(NotSafePosPrompt)) + " images will be generated - NSFW")
for row in range(len(NotSafePosPrompt)):
    data = {'prompt': NotSafePosPrompt.at[row, 'prompt'],
            'negative_prompt': "(worst quality, low quality:1.4), monochrome, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry, deformed eyes, ((disfigured)), bad art, deformed, ((extra limbs)), ((duplicate)), morbid, multilated, bad body, on hand with less than 5 fingers, crown , stacked torses, stacked hands, totem pole",
            'width': 512,
            'height': 512,
            'steps': 50}
    response = submit_post(txt2img_url, data)
    fullpath = save_encoded_image(response.json()['images'][0], "classes/new_pictures")
    isNSFW = nsfw_detector("./" + full_path)
    if isNSFW:
        shutil.move(full_path, "classes/nsfw")
    else:
        shutil.move(full_path, "classes/sfw")
    print(row , end='->')

100 images will be generated - NSFW


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\rybak\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-05c9b8348bf4>", line 11, in <module>
    isNSFW = nsfw_detector("./" + full_path)
NameError: name 'full_path' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\rybak\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\rybak\appdata\local\programs\python\python37\lib\site-packages\IPython\core\ultratb.py", line 1169, in get_records
    return _fixed_getinnerfr